In [9]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import cv2
%matplotlib inline

In [10]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            
def get_points(fitted_line,image_shape):
    """
    This function returns the two points (x1,y1) , (x2,y2) to draw the fitted line using parameters (m,b)
    """
    #print(image_shape)
    slope,intersect = fitted_line
    # first vertical point start @ the end of the image
    y1 = image_shape[0] # get the hight of image which represents the buttom
    
    y2 = 330  #ithink its enough to draw lines till the half of the image
    
    # using the equation of line y = m*x + b we can get x values of y1 & y2
    # m = fitted_line[0] , b = fitted_line[1]
    x1 = int ((y1-intersect) / (slope))
    x2 = int ((y2-intersect) / (slope))
    
    return np.array([x1,y1,x2,y2])


def line_seperation(lines,image):
    """
    This function seprates 'lines' ie : seperate left and right lane lines
    The key concept is that the slope of the left lane line decreasing and the slope of the right lane line increasing
    
    """

    left_lane_x = []  # left segment x-points
    left_lane_y = []  # left segment y-points

    right_lane_x = [] # right segment x-points
    right_lane_y = [] # right segment y-points   
    for line in lines:
        for x1,y1,x2,y2 in line:
            
            
            if x2 == x1:
                continue
            slope = 0.1*(y2-y1)/(x2-x1)   # get the slope of the line (m)
            
            if slope < 0 : # left segment as the slope decreasing
                left_lane_x.append(x1)
                left_lane_x.append(x2)
                left_lane_y.append(y1)
                left_lane_y.append(y2)
            if slope > 0 : # right segment as the slope increasing
                right_lane_x.append(x1)
                right_lane_x.append(x2)
                right_lane_y.append(y1)
                right_lane_y.append(y2)

    left_lane_x = np.array(left_lane_x)
    left_lane_y = np.array(left_lane_y)
    
    right_lane_x = np.array(right_lane_x)
    right_lane_y = np.array(right_lane_y)
    
    line_parameters_left  =   np.polyfit(left_lane_x,left_lane_y,1)     
    line_parameters_right =   np.polyfit(right_lane_x,right_lane_y,1)     

    left_fit  = get_points(line_parameters_left,image.shape)
    right_fit = get_points(line_parameters_right,image.shape)
    
    return np.array([[left_fit],[right_fit]])
   

def get_line_parameters(line):
    """
    This function gets the parameters of line ( slope(m) and intersection(b))
    """
    # seperate x points from y points 
    x_points = []
    y_points = []
    for points in line:
        for x1,y1,x2,y2 in points:
            x_points.append(x1)
            x_points.append(x2)
            y_points.append(y1)
            y_points.append(y2)
    
    x_points = np.array(x_points)
    y_points = np.array(y_points)
    
    # get the best fit line     
    m, b = np.polyfit(x_points, y_points, 1)
    
    best_fit_line = m*x_points+b

    row = []
    result = []
    i=1
    for x,y in zip(x_points, best_fit_line):
        row.append(x)
        row.append(int(y))
        
        if i == 2:
            i=0
            result.append([row])
            row = [] # clear list
        i +=1     

    result = np.array(result)
    #print("drawing best fit line")
    #plt.plot(x_points,best_fit_line)
    #plt.show()
    return result    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    lanes = line_seperation(lines,img)
    #print("lanes shape ",lanes.shape)
    #print("lanes[0][0]",np.array([[lanes[0][0]]]) )
    draw_lines(line_img,lanes)  
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)


def get_roi_vertecies(image_shape):
    """
    This funtion tries to estimate the best vertecies for ROI depend on the image shape
    """
    width = image_shape[1]   # width of the image 
    height = image_shape[0]   # height of the image
    
    # we will divide image into two sections (right and left)
    sepration_point_x = width/2
    sepration_point_y = height/2
    
    upper_right_v_x = sepration_point_x + 0.25*sepration_point_x
    upper_left_v_x  = sepration_point_x - 0.25*sepration_point_x
    
    upper_right_v_y = 0.25*height
    upper_left_v_y  = 0.25*height   
    
    lower_right_v_x = sepration_point_x + 0.90*sepration_point_x
    lower_left_v_x  = sepration_point_x - 0.80*sepration_point_x
    
    lower_right_v_y = height
    lower_left_v_y  = height
    
    upper_left_pt = [upper_left_v_x,upper_left_v_y]
    upper_right_pt = [upper_right_v_x,upper_right_v_y]
    
    lower_left_pt = [lower_left_v_x,lower_left_v_y]
    lower_right_pt = [lower_right_v_x,lower_right_v_y]
    
    vertcies = np.array([upper_left_pt,upper_right_pt,lower_left_pt,lower_right_pt
                        ])
    
    return vertcies

In [72]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images_output directory.
# first read the image 
# convert image to gray scale
def pipline(image):
    image_gray = grayscale(image)
    #print("original image")
    #plt.imshow(image_gray,cmap='gray')  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')
    #plt.show()
    
    # apply gaussian filter to image
    image_blur = gaussian_blur(image_gray,5)
    #print("Blured image")
    #plt.imshow(image_blur,cmap='gray')
    #plt.show()

    # create canny edge detection to detect edges
    image_edges = canny(image_blur,100,300)
    #print("Edges image")
    #plt.imshow(image_edges,cmap='gray')
    #plt.show()
    
    # apply ROI to get the region we interested 
    # left , right , vertex
    #vertcies = np.array([[130,540],[900,540],[480,290]])
    vertcies = get_roi_vertecies(image.shape)
    roi = region_of_interest(image_edges,np.int32([vertcies]))
    #print("ROI image")
    #plt.imshow(roi,cmap='gray')
    #plt.show()



    # detect lines using hough transform
    image_lines = hough_lines(roi, 1, np.pi/180,35, 1, 1)
    
    #lines_index = np.where(image_lines != 0)
    #plt.scatter(lines_index[1],lines_index[0])
    #plt.show()
    #print("HOUGH image")
    #plt.imshow(image_lines,cmap='gray')
    #plt.show()
    
    # using dilation and erotion to connect lines
    kernel = np.ones((1,8), np.uint8)  # note this is a horizontal kernel
    image_lines = cv2.dilate(image_lines, kernel, iterations=2)
    #e_im = cv2.erode(d_im, kernel, iterations=1    
    #print("Dilate image")
    #plt.imshow(image_lines,cmap='gray')
    #plt.show()
    
    image_weighted =  weighted_img(image_lines, image, α=0.8, β=1., γ=0.)
    #plt.imshow(image_weighted,cmap='gray')
    #plt.show()
    return image_weighted

In [73]:
test_images = os.listdir("test_images/")
image = mpimg.imread('test_images/'+test_images[0])
pipline(image)


array([[[ 87, 126, 159],
        [ 87, 126, 159],
        [ 87, 126, 159],
        ...,
        [ 89, 125, 158],
        [ 86, 122, 155],
        [ 82, 118, 151]],

       [[ 87, 126, 159],
        [ 87, 126, 159],
        [ 87, 126, 159],
        ...,
        [ 89, 125, 158],
        [ 86, 122, 155],
        [ 83, 119, 152]],

       [[ 87, 126, 159],
        [ 87, 126, 159],
        [ 87, 126, 159],
        ...,
        [ 89, 125, 158],
        [ 86, 122, 155],
        [ 83, 119, 152]],

       ...,

       [[ 68,  70,  76],
        [ 68,  70,  76],
        [ 68,  70,  76],
        ...,
        [ 65,  65,  73],
        [ 64,  64,  72],
        [ 63,  63,  71]],

       [[ 69,  69,  75],
        [ 69,  69,  75],
        [ 70,  70,  76],
        ...,
        [ 62,  64,  71],
        [ 62,  64,  71],
        [ 61,  63,  70]],

       [[ 67,  67,  74],
        [ 67,  67,  74],
        [ 67,  67,  74],
        ...,
        [ 61,  62,  72],
        [ 60,  62,  71],
        [ 59,  61,  70]]

In [74]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [75]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    result = pipline(image)
    return result

In [78]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)


                                                                                                                    
t:   8%|█████▌                                                              | 18/221 [02:36<00:03, 54.41it/s, now=None]
                                                                                                                    
t:   1%|▌                                                                    | 2/221 [00:00<00:12, 17.86it/s, now=None]

Moviepy - Building video test_videos_output/solidWhiteRight.mp4.
Moviepy - Writing video test_videos_output/solidWhiteRight.mp4



                                                                                                                       
                                                                                                                    
t:   8%|█████▌                                                              | 18/221 [02:39<00:03, 54.41it/s, now=None]
                                                                                                                    
t:   8%|█████▌                                                              | 18/221 [02:39<00:03, 54.41it/s, now=None]

Moviepy - Done !
Moviepy - video ready test_videos_output/solidWhiteRight.mp4
Wall time: 2.88 s


In [79]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [80]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)


                                                                                                                    
t:   8%|█████▌                                                              | 18/221 [07:08<00:03, 54.41it/s, now=None]
                                                                                                                    
t:   0%|▎                                                                    | 3/681 [00:00<00:25, 27.03it/s, now=None]

Moviepy - Building video test_videos_output/solidYellowLeft.mp4.
Moviepy - Writing video test_videos_output/solidYellowLeft.mp4



                                                                                                                       
                                                                                                                    
t:   8%|█████▌                                                              | 18/221 [07:16<00:03, 54.41it/s, now=None]
                                                                                                                    
t:   8%|█████▌                                                              | 18/221 [07:16<00:03, 54.41it/s, now=None]

Moviepy - Done !
Moviepy - video ready test_videos_output/solidYellowLeft.mp4
Wall time: 7.73 s


In [81]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [82]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)


                                                                                                                    
t:   8%|█████▌                                                              | 18/221 [13:12<00:03, 54.41it/s, now=None]
                                                                                                                    
t:   2%|█                                                                    | 4/251 [00:00<00:06, 35.71it/s, now=None]

Moviepy - Building video test_videos_output/challenge.mp4.
Moviepy - Writing video test_videos_output/challenge.mp4



t:  44%|█████████████████████████████▎                                     | 110/251 [00:02<00:03, 42.59it/s, now=None]

TypeError: expected non-empty vector for x

t:  44%|█████████████████████████████▋                                     | 111/251 [00:12<00:03, 42.59it/s, now=None]